# Project 1 – Scrape an ICT variable

Obtaining an ICT variable, namely social media presence by web scraping and comparing it to the TurkStat ICT Usage in Enterprises survey results.

The following code needs a text file named "url.txt", which consists of listing web addresses one after the other, as input.

I visited Jacek Maślankowski (Ph.D., Assistant Professor, University of Gdańsk, Poland) WP2-Social-Media-Presence GitHub repository and examined the code he wrote and adapted the relevant code for TurkStat case. The application is used to scrap all the links related to social media from websites (https://github.com/jmaslankowski/WPC-Social-Media-Presence/blob/master/WP2_SocialMediaPresence_Dev.py).

In [1]:
import requests
import re
import string
import csv
import sys
import json
import os
import os.path
from bs4 import BeautifulSoup 
from collections import OrderedDict
from datetime import datetime


# Global variable to store the list for JSON purposes
jsonList=[]

# HTMLParserBS class is used to find all URLs that reference to Social Media
class HTMLParserBS:    
    output_list=[]
    # method extractURLs is to extract all URLs and return them as the urls[] list
    # it goes through the website to find all anchors <a href>
    def extractURLs(self,page):      
        # all links are lowered because sometimes the same link is written differently, e.g., ug.edu.pl or UG.edu.pl
        soup = BeautifulSoup(page.text.lower(),"html.parser")
        urls = []
        for a in soup.find_all('a', href=True):
            urls.append(a['href'])
        return urls
    # method extractsURLs is to extract all URLs that are not in anchors <a>
    # it uses regular expression that search for any http or https, even not included in anchors
    def extractAllHTTP(self,page):
        URLs=re.findall(r"https?://[\w\-.~/?:#\[\]@!$&'()*+,;=]+", page.text.lower())
        return URLs

# class SocialMediaDeep contains a function responsible to divide the links into:
# (1) internal (used for the second search and external)
# (2) external (not used for the second search of social media)
class SocialMediaDeep:
    website=""    
    # this method is responsible to do the second search on subpages
    # from internal links that are present on the main page of the website
    def goDeeperToFindSocialMedia(self,website,URLs):
        print("Preparing to scrape subpages...")
        for url in URLs:
            try:        
                if url:
                    # the difference between InternalURL_type2 and InternalURL_type1
                    # is that type2 includes the domain name of the website, e.g., http://stat.gov.pl/page2.html
                    # and type1 does not include the domain name, e.g., <a href="./links/page2.html" ...>
                    if website in url and 'javascript' not in url:
                        print("Scraping InternalURL_type2: %s" % url)
                        smp.searchSocialMediaLinks(url,'2')
                    elif url[0]=="/" or url[0:1]=="./" or "http" not in url:
                        if url[0]=="/":
                            print("Scraping InternalURL_type1: {0} {1} ".format(website,url))
                            smp.searchSocialMediaLinks(website+url,'2')
                        elif url[0]==".":
                            print("Scraping InternalURL_type1: {0} {1}".format(website,url.replace('./','/')))
                            smp.searchSocialMediaLinks(website+url.replace('./','/'),'2')
                        else:
                            print("Scraping InternalURL_type1: {0} {1} {2}".format(website,'/',url))
                            smp.searchSocialMediaLinks(website+'/'+url,'2')
                    else:
                        # all external URLs are those who have a domain name different than the main page
                        print("ExternalURL_type1: %s " % url)                    
                else:
                    print("URL was not found in the second search.")
            except:
                print("Exception occured during processing the following URL:"+url)

class FileAccess:
    def jsonListWrite(self,jsonList): 
        currentDate = re.sub('[- :.]', '', str(datetime.now()))
        try:
            with open("wp2_social_"+currentDate+".json", "w") as file:
                file.write(str(jsonList))
                file.close()
        except IOError:
            msg = ("Error writing JSON file.")     
            print(msg)
            return

# class SocialMediaPresence includes one function responsible for finding popular Social Media websites
class SocialMediaPresence:
    website=""
    def __init__(self, social_media_dict={
                                    'Facebook': ['facebook.com'],
                                     'Twitter': ['twitter.com'],
                                     'Youtube': ["youtu.be", "youtube.com"],
                                     'LinkedIn': ["linkedin.com"],
                                     'Instagram': ["instagram.com"],
                                     'Xing':['xing.com'],
                                     'Pinterest': ['pinterest.com']} ):
        # social_media_dict specifies which social media links should be found
        self.social_media_dict = social_media_dict
     
    # this method finds all social media links on webpage
    def searchSocialMediaLinks(self,website,level):        
        try:
            headers = {'user-agent': 'python-app/0.1 experimental for statistical purposes'}
            r = requests.get(website, headers=headers)            
        except:    
            print("Exception during scraping content of the webpage: "+website)
        else:
            print("The length of the scrapped content: %s characters" % str(len(r.text)))            
            rows=r.text.splitlines()
            # if facebook login button is present on the website - report this on the screen
            # not used now but maybe in the future it can enrich the ICT survey
            #for line in rows:
            #    if "facebookLoginButton" in line:
            #        print ("Facebook login found: %d" % line)
            p = HTMLParserBS()
            p.output_list=p.extractURLs(r)
            URLs=list(p.output_list)
            print("Number of links on website: %d" % len(URLs))
            # sets are used instead of lists to eliminate all duplicates automatically
            # sometimes inside the main page there are several links to Social Media, in this case all duplicates will be removed
            # but all Social Media links will be added to the final list
            # Dictionary to store links for specific social media platforms
            link_dict = {}
            link_dict['URL'] = [website]
            for social_media in self.social_media_dict:
                link_dict[social_media] = set([])
            none='';
            # this loops through through all specified social media domain names
            for url in URLs:
                if url:
                    for social_media in self.social_media_dict:
                        for domain in self.social_media_dict[social_media]:
                            if domain in url:
                                link_dict[social_media].add(url)
                                print(url)
           # Count the number of found links
            n_links = 0
            for social_media in self.social_media_dict:
                n_links += len(link_dict[social_media])

            if n_links==0:
                print('No social media links have been found.')
                none='1';
            else:
                print('Total number of unique social media links found:',
                      n_links)
            # if subpage does not have a social media URL - do not write this in the result file
            if not (none=='1' and level=='2'):            
                try:
                    # name of the file
                    filename='wp2_social.csv'
                    exists=1
                    if not os.path.isfile(filename):
                        exists=0
                    with open (filename,'a') as file:
                        if exists==0:
                            file.write(';'.join(link_dict) + "\n")
                        columnNames= list(link_dict.keys())                    
                        writer=csv.DictWriter(file,delimiter=';',dialect=csv.excel,fieldnames=columnNames)
                        writer.writerow({item: ' ,'.join(link_dict[item]) for item in link_dict})
                except IOError:
                    msg = ("Error writing CSV file.")     
                    print(msg)      
                data = {}
                data['URL'] = website
                for social_media in self.social_media_dict:
                    if social_media in link_dict:
                        data[social_media] = list(link_dict[social_media])
                    else:
                        data[social_media] = []
                jsonList.append(data);
            smd=SocialMediaDeep();
            # if no links have been found on the main page, go one level deeper
            if none=='1' and level=='1' and len(URLs)>1:
                smd.goDeeperToFindSocialMedia(website,URLs)
  
smp=SocialMediaPresence()
fa=FileAccess()        
jsonList=[]
try:
    plik=open("url.txt","r")
except IOError:
    msg = ("Error reading URL file.\n\nPlease create a file named 'url.txt' in the current folder with the list of URLs you want to scrap.")
    msg += ("\n\nThe file content should be like this (one URL per line):\nmaslankowski.pl\nhttp://www.stat.gov.pl")
    print(msg)
else:
    for url in plik:
        if url!="":
            website='http://'+url.strip().replace('http://','').replace('https://','')
            print("Website currently being scrapped: "+website)
            smp.searchSocialMediaLinks(website,'1')
            print()
    fa.jsonListWrite(jsonList)

Website currently being scrapped: http://www.fieldcore.com/
Exception during scraping content of the webpage: http://www.fieldcore.com/

Website currently being scrapped: http://www.aytemiz.com.tr
Exception during scraping content of the webpage: http://www.aytemiz.com.tr

Website currently being scrapped: http://tr.panasonic.com
Exception during scraping content of the webpage: http://tr.panasonic.com

Website currently being scrapped: http://www.oyka.com.tr
The length of the scrapped content: 25600 characters
Number of links on website: 66
https://www.youtube.com/embed/-6kye1vqufo
https://twitter.com/oykakagit
https://www.facebook.com/oykakagit
https://www.linkedin.com/company/417304?trk=tyah&trkinfo=clickedvertical%3acompany%2cclickedentityid%3a417304%2cidx%3a1-1-1%2ctarid%3a1454429310527%2ctas%3aoyka
Total number of unique social media links found: 4

Website currently being scrapped: http://www.dolu.com.tr
The length of the scrapped content: 337105 characters
Number of links on we

The length of the scrapped content: 96699 characters
Number of links on website: 29
No social media links have been found.
Scraping InternalURL_type2: http://www.gyhmetro.com/teknik-bilgiler/isin-kapsami/
The length of the scrapped content: 95999 characters
Number of links on website: 28
No social media links have been found.
Scraping InternalURL_type2: http://www.gyhmetro.com/gorseller/
The length of the scrapped content: 95339 characters
Number of links on website: 28
No social media links have been found.
Scraping InternalURL_type2: http://www.gyhmetro.com/gorseller/video-galeri/
The length of the scrapped content: 95533 characters
Number of links on website: 28
No social media links have been found.
Scraping InternalURL_type2: http://www.gyhmetro.com/gorseller/fotograf-galerisi/
The length of the scrapped content: 106633 characters
Number of links on website: 39
No social media links have been found.
Scraping InternalURL_type2: http://www.gyhmetro.com/iletisim/
The length of the sc

The length of the scrapped content: 338311 characters
Number of links on website: 48
No social media links have been found.
Scraping InternalURL_type1: http://www.lilakagit.com /haberler?id=17 
The length of the scrapped content: 338311 characters
Number of links on website: 48
No social media links have been found.
Scraping InternalURL_type1: http://www.lilakagit.com /haberler?id=34 
The length of the scrapped content: 338311 characters
Number of links on website: 48
No social media links have been found.
Scraping InternalURL_type1: http://www.lilakagit.com /haberler/ 
The length of the scrapped content: 338300 characters
Number of links on website: 48
No social media links have been found.
ExternalURL_type1: http://www.belgemodul.com/sirket/229 

Website currently being scrapped: http://www.doluca.com
Exception during scraping content of the webpage: http://www.doluca.com

Website currently being scrapped: http://arikan@arikan.com.tr
Exception during scraping content of the webpage: 

The length of the scrapped content: 30026 characters
Number of links on website: 73
No social media links have been found.
Scraping InternalURL_type2: http://www.greif.com.tr/default.asp?p=0&l=2&k=0&k1=9
The length of the scrapped content: 25584 characters
Number of links on website: 70
No social media links have been found.
Scraping InternalURL_type2: http://www.greif.com.tr/default.asp?p=0&l=2&k=0&k1=10
The length of the scrapped content: 31671 characters
Number of links on website: 70
No social media links have been found.
Scraping InternalURL_type2: http://www.greif.com.tr/default.asp?p=0&l=2&k=0&k1=16
The length of the scrapped content: 29011 characters
Number of links on website: 72
No social media links have been found.
Scraping InternalURL_type2: http://www.greif.com.tr/default.asp?p=0&l=2&k=0&k1=17
The length of the scrapped content: 26584 characters
Number of links on website: 70
No social media links have been found.
Scraping InternalURL_type2: http://www.greif.com.tr/defaul

The length of the scrapped content: 26584 characters
Number of links on website: 70
No social media links have been found.
Scraping InternalURL_type1: http://www.greif.com.tr/ / default.asp?k=0&k1=13
The length of the scrapped content: 26593 characters
Number of links on website: 73
No social media links have been found.
Scraping InternalURL_type1: http://www.greif.com.tr/ / default.asp?k=0&k1=14
The length of the scrapped content: 26364 characters
Number of links on website: 72
No social media links have been found.
Scraping InternalURL_type1: http://www.greif.com.tr/ / default.asp?k=0&k1=15
The length of the scrapped content: 25505 characters
Number of links on website: 71
No social media links have been found.
Scraping InternalURL_type1: http://www.greif.com.tr/ / default.asp?k=0&k1=12
The length of the scrapped content: 29730 characters
Number of links on website: 71
No social media links have been found.
Scraping InternalURL_type1: http://www.greif.com.tr/ / default.asp?k=0&k1=13


...

The length of the scrapped content: 17332 characters
Number of links on website: 84
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/enerji-satis/sikca-sorulan-sorular 
The length of the scrapped content: 15450 characters
Number of links on website: 84
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/enerji-satis/indirimli-elektrik-satisi 
The length of the scrapped content: 80439 characters
Number of links on website: 85
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/online-islemler 
The length of the scrapped content: 14042 characters
Number of links on website: 92
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/haberler/bizden-haberler 
The length of the scrapped content: 32050 characters
Number of links on website: 117
No social media links have been found.
Scraping Internal

The length of the scrapped content: 15404 characters
Number of links on website: 83
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/kariyer/insan-kaynaklari-politikamiz 
The length of the scrapped content: 15404 characters
Number of links on website: 83
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/kariyer/acik-pozisyonlar 
The length of the scrapped content: 13159 characters
Number of links on website: 84
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/iletisim 
The length of the scrapped content: 21693 characters
Number of links on website: 103
No social media links have been found.
Scraping InternalURL_type1: http://www.eren-enerji.com.tr /tr/iletisim/adres-bilgilerimiz 
The length of the scrapped content: 21693 characters
Number of links on website: 103
No social media links have been found.
Scraping InternalURL_type1: http

The length of the scrapped content: 19432 characters
Number of links on website: 33
No social media links have been found.
URL was not found in the second search.
Scraping InternalURL_type1: http://www.dinateks.com /pages/about-us.html 
The length of the scrapped content: 13526 characters
Number of links on website: 33
No social media links have been found.
Scraping InternalURL_type1: http://www.dinateks.com /factory/ 
The length of the scrapped content: 14913 characters
Number of links on website: 33
No social media links have been found.
Scraping InternalURL_type1: http://www.dinateks.com /contact/ 
The length of the scrapped content: 16516 characters
Number of links on website: 33
No social media links have been found.
ExternalURL_type1: https://www.renklikalem.com 

Website currently being scrapped: http://www.olmuksan-ipaper.com
Exception during scraping content of the webpage: http://www.olmuksan-ipaper.com

Website currently being scrapped: http://www.hataymedeniyetlersofrasi.co

Exception during scraping content of the webpage: http://www.strom.com.tr

Website currently being scrapped: http://www.avva.com.tr/
Exception during scraping content of the webpage: http://www.avva.com.tr/

Website currently being scrapped: http://esasguvenlik.com/
The length of the scrapped content: 86648 characters
Number of links on website: 81
https://www.facebook.com
http://instagram.com/
Total number of unique social media links found: 2

Website currently being scrapped: http://www.habas.com.tr
Exception during scraping content of the webpage: http://www.habas.com.tr

Website currently being scrapped: http://www.dpworld.com.tr
Exception during scraping content of the webpage: http://www.dpworld.com.tr

Website currently being scrapped: http://www.grupgokkusagi.com.tr
The length of the scrapped content: 13029 characters
Number of links on website: 37
https://www.facebook.com/grupgokkusagihipermarket/
https://twitter.com/grup_gokkusagi
https://www.instagram.com/grup.gokkusagi
Tot

The length of the scrapped content: 106490 characters
Number of links on website: 90
No social media links have been found.
Scraping InternalURL_type1: http://www.tdp.com.tr / hakkimizda/kalite-politikamiz
The length of the scrapped content: 15093 characters
Number of links on website: 43
No social media links have been found.
Scraping InternalURL_type1: http://www.tdp.com.tr / bize-ulasin
The length of the scrapped content: 20945 characters
Number of links on website: 43
No social media links have been found.
Scraping InternalURL_type1: http://www.tdp.com.tr / haberler
The length of the scrapped content: 15378 characters
Number of links on website: 43
No social media links have been found.
Scraping InternalURL_type1: http://www.tdp.com.tr / informative-text
The length of the scrapped content: 25895 characters
Number of links on website: 47
No social media links have been found.
Scraping InternalURL_type1: http://www.tdp.com.tr / cookies-policy
The length of the scrapped content: 26670